# Song Recommender App

In [60]:
from config import *
import spotipy 
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [61]:
import pickle
from sklearn.cluster import KMeans

In [62]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [22]:
db_songs = pd.read_csv('final_df_songs.csv')
db_songs.head()

,Unnamed: 0,title,artist,id,danceability,energy,key,loudness,mode,speechiness,...,type,id.1,uri,track_href,analysis_url,duration_ms,time_signature,original_df,kmean_cluster,db_cluster
0,0,Abere izate ukatua,Su ta gar,4tMbI2qkOIfc1kzsbw7X64,0.393,0.967,10.0,-5.775,0.0,0.0511,...,audio_features,4tMbI2qkOIfc1kzsbw7X64,spotify:track:4tMbI2qkOIfc1kzsbw7X64,https://api.spotify.com/v1/tracks/4tMbI2qkOIfc...,https://api.spotify.com/v1/audio-analysis/4tMb...,243973.0,4.0,not_hot,1,0
1,1,Viva Forever (Tony Rich Remix Instrumental),Spice Girls,1dehm5fgmBVxxur0YnZLMq,0.809,0.478,10.0,-7.371,0.0,0.1870,...,audio_features,1dehm5fgmBVxxur0YnZLMq,spotify:track:1dehm5fgmBVxxur0YnZLMq,https://api.spotify.com/v1/tracks/1dehm5fgmBVx...,https://api.spotify.com/v1/audio-analysis/1deh...,342733.0,4.0,not_hot,6,1
2,2,Think About That,Dandy Livingstone,5vWscUwJJSsIvKR0YJa8cV,0.759,0.654,0.0,-8.462,1.0,0.0523,...,audio_features,5vWscUwJJSsIvKR0YJa8cV,spotify:track:5vWscUwJJSsIvKR0YJa8cV,https://api.spotify.com/v1/tracks/5vWscUwJJSsI...,https://api.spotify.com/v1/audio-analysis/5vWs...,172947.0,4.0,not_hot,3,0
3,3,That Groovy Shingaling,Orquesta Harlow,2kTZKa7J0ykBpmzs5TMjWT,0.454,0.481,10.0,-10.895,1.0,0.0318,...,audio_features,2kTZKa7J0ykBpmzs5TMjWT,spotify:track:2kTZKa7J0ykBpmzs5TMjWT,https://api.spotify.com/v1/tracks/2kTZKa7J0ykB...,https://api.spotify.com/v1/audio-analysis/2kTZ...,135187.0,4.0,not_hot,2,0
4,4,Take It Off,King B.A.V.,77ihANWN9MuLGim3ZtYIti,0.706,0.580,7.0,-3.692,0.0,0.0423,...,audio_features,77ihANWN9MuLGim3ZtYIti,spotify:track:77ihANWN9MuLGim3ZtYIti,https://api.spotify.com/v1/tracks/77ihANWN9MuL...,https://api.spotify.com/v1/audio-analysis/77ih...,244827.0,4.0,not_hot,1,-1


In [77]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File not found!")
        

        
def ask_user(results):
    
    artists_list = []
    songs_list = []
    for i in range(len(results["tracks"]["items"])):
        artists_list.append(results["tracks"]["items"][i]['artists'][0]['name'])
        songs_list.append(results["tracks"]["items"][i]['name'])

    display(pd.DataFrame({'song':songs_list,'artist':artists_list}))

    input_song = input('To which number corresponds your favorite song ? ')

    return results["tracks"]["items"][int(input_song)]["id"]



def get_cluster(song_id):
    audio_features = sp.audio_features(song_id)
    audio_features_df = pd.DataFrame(audio_features)
    audio_features_for_model = audio_features_df.drop(['id', 'key', 'mode', 'type', 'uri', 'track_href', 
                                    'analysis_url', 'duration_ms', 'time_signature'], axis = 1)
    
    scaler = load("/Users/macbook/Documents/IronHack/Functions_saved/Models/standardscaler_9_features.pickle")
    audio_features_scaled = scaler.transform(audio_features_for_model)
    audio_features_scaled_df = pd.DataFrame(audio_features_scaled, columns = audio_features_for_model.columns)
    #until here it works
    
    # Our best model was k-means for k = 14
    best_model = load("/Users/macbook/Documents/IronHack/Functions_saved/Models/k_means/kmeans_14.pickle")
    return best_model.predict(audio_features_scaled_df.values)



def song_recommender() :
    
    #ask for input from the user
    x = input("Please enter your favorite song : ")
    
    # get user's song_id
    results = sp.search(q="track:"+x,limit=10)
    results_size = len(results["tracks"]["items"])
    if results_size == 0 :
        print("We didn't find your song :(")
    elif results_size == 1 :
        song_id = results["tracks"]["items"][0]["id"]
    else : 
        song_id = ask_user(results)
           
    # get audio features and predict cluster membership of the user's song
    cluster = get_cluster(song_id)
    
    # recommend a song to the user
    if song_id in list(db_songs[db_songs['original_df'] == 'hot']['id']) :
        results_top_100 = list(np.random.choice(db_songs[(db_songs['original_df'] == 'hot') & (db_songs['kmean_cluster'] == cluster[0])]['title'], 3))
        #results_df_top_100 = db_songs[db_songs['title'] in results_list_top_100]
        print()
        print("You should try the following tracks ;) :")
        print()
        for i in range(len(results_top_100)) :
            print((results_top_100[i]+" by "+db_songs[db_songs['title'] == results_top_100[i]]['artist']).to_frame().iloc[0,0])
    else : 
        results_not_hot = list(np.random.choice(db_songs[(db_songs['original_df'] == 'not_hot') & (db_songs['kmean_cluster'] == cluster[0])]['title'], 3))
        print()
        print("You should try the following tracks ;) :")
        print()
        for i in range(len(results_not_hot)) :
            print((results_not_hot[i]+" by "+db_songs[db_songs['title'] == results_not_hot[i]]['artist']).to_frame().iloc[0,0])
    
    # another round ?
    print()
    last = input("You want more :) ? 0 = No // 1 = Yes ")
    if last == "1" :
        print()
        song_recommender()
    else :
        print()
        print("Bye bye and see you next time ! Don't hesitate to share this recommender around you ;) ")

        
    


In [78]:
song_recommender()

Please enter your favorite song : forever monna


,song,artist
0,Forever Monna,Chez Damier
1,Forever 28,Monna
2,Forever Jung (The Self),Monna


To which number corresponds your favorite song ? 0

You should try the following tracks ;) :

Discretion Grove by Stephen Malkmus
Victims Among Friends (Amduscia Mix) by God Module
Booty Pills by Les Petits Pilous

You want more :) ? 0 = No // 1 = Yes 0

Bye bye and see you next time ! Don't hesitate to share this recommender around you ;) 
